In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
assert x_train.shape == (50000, 32, 32, 3)
assert x_test.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 1)
assert y_test.shape == (10000, 1)

In [3]:
def extractor(inputs):
  extracting_layer =  tf.keras.applications.resnet.ResNet50(input_shape=(224,224,3),
                                                            include_top=False,weights='imagenet')(inputs)
  return extracting_layer

In [4]:
def classifier(inputs):
  x = tf.keras.layers.GlobalAveragePooling2D()(inputs)
  x = tf.keras.layers.Flatten()(x)
  x = tf.keras.layers.Dense(1024,activation='relu')(x)
  x = tf.keras.layers.Dense(512,activation='relu')(x)
  x = tf.keras.layers.Dense(10,activation='softmax',name='classification')(x)
  return x

In [5]:
def final_model(inputs):

  resize = tf.keras.layers.UpSampling2D(size=(7,7))(inputs)
  get = extractor(resize)
  model = classifier(get)
  return model

In [10]:
def define_compile_model():
  inputs = tf.keras.layers.Input(shape=(32,32,3))
  
  classification_output = final_model(inputs) 
  model = tf.keras.Model(inputs=inputs, outputs = classification_output)
 
  model.compile(optimizer='adam', 
                loss='sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  
  return model

model = define_compile_model()

In [ ]:
history = model.fit(x_train, y_train, epochs=5,validation_split=0.30, batch_size=64)